# Overview

## XGBoost, LightGBMとは？

gradient boosting をC++で実装したものがXGBoost であり、それをベースにしてより軽量な実装を行ったものが LightGBMである。どちらも実装面での話であるため、以下で議論する Gradient Boosting Tree (GBT) を理解すればok。

# XGBoost

- https://arxiv.org/pdf/1603.02754.pdf
- https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/
- https://xgboost.readthedocs.io/en/latest/tutorials/model.html
- https://towardsdatascience.com/xgboost-mathematics-explained-58262530904a

## XGBoostとは？

スケーラビリティの高い boosting 用の学習アルゴリズムである。

## Boosting について

アンサンブル学習の一つで、一つのデータから一つのモデルを一回きり作成するのではなく、weak leaner (wark classifier, 弱学習器）を何度も作成し、それらの多数決で最終的な予測を行うものである。boosted tree はいわゆる直列で tree を作成するモデルであり（あるイテレーションで作成した tree が次のイテレーションの tree の判断に影響を与える）、bagging とは異なる概念を持つ。歴史的には以下のようになっている。

- Random Forest (Breiman, 1997)
- Gradient Tree Boosting (Friedman, 1999)
- Gradient Tree Boosting with Regulatization --> XGBoost

tree 系は精度がよく、使いやすく、解釈しやすいというメリットがある一方で、すぐに過学習してしまうし、学習スピードに難があった。

### AdaBoost
### XGBoost

# Decision Tree Ensamble

サンプル数$n$、特徴量個数 $m$ のデータが与えられたとする。

$$
\mathcal{D}=\{(\boldsymbol{x}_i, y_i)\}~~,\text{where}~|\mathcal{D}|=n,~\boldsymbol{x}_i\in R^m,~y_i\in R
$$

ここで、$K$ 本のツリーを作成して、それらを用いて予測値を算出する場合を考える（ex. 特徴量全てを使うのではなく、$K$通りの組み合わせでツリーを作るとか）。tree ensamble とは各ツリーの結果の線形結合で一つの予測値を算出することを指す。

$$
\hat{y}_i = \sum_{k=1}^K f_k(x_i),~~f_k \in \mathcal{F}
$$

この場合の目的関数は

$$
\text{obj} = \sum_{i=1}^n l(y_i,\hat{y}_i) + \sum_{k=1}^K \Omega(f_k)
$$

第一項は損失関数であり、第二項は正則化項でモデルが複雑にならないように調整する役割を果たす。ツリー系は前述の通り over fitting しやすいので正則化項が必要だが、もちろんそこにはトレードオフが存在している。

- 損失関数を最適化することは、予測性能の高いモデルを組むことに相当する
- 正則化項を最適化することは、シンプルなモデルを組むことに相当する。
    - シンプル = 予測性能が悪い、ではなく、予測が安定し（例えば outlier に反応しない）信頼のおけるモデルであるという意味を含む。
    
    
ここでモデルを表現するための量を定義する。

$$
f_t(x) = w_{q(x)},~~w \in R^T,~q : R^d \to \{1,2,...,T\}
$$


# Tree Boosting

目的関数は以下のように再定義する：

$$
\text{obj} = \sum_{i=1}^n l(y_i, \hat{y}_i^{(t)}) + \sum_{i=1}^t \Omega(f_i)
$$

ここで添え字 $t$ はステップ(or イテレーション回数）を表し、$t$ 回目にツリーを作成するときの目的関数である。

## Additive Training

ツリーを学習するときに、そのツリーの構造もパラメータで調整できるが、これは目的関数から最小化するのは困難である。ツリーの構造は即ちモデルの構造であり（DNNであればレイヤー数やレイヤーの種類）、それをイテレーション毎に過去に遡って最適化することは難しい。そこで、additive training （加法性学習）という形で、$t-1$ までで組んだツリーに新たに一本ツリーを追加して、それを持って予測値と目的関数を計算するというものである。つまり、過去までの学習過程を踏まえて現時点でどんなツリーが好ましいかを学習するのである（=直列性）。またこれは、通常の損失関数の最小化ともリンクしており、通常も多変数関数である損失関数を一気に最小化することは困難である。そこで損失関数の勾配を使って（勾配降下法）、最適化を行っているが、それと類似している。

$$
\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i) = \sum_{k=1}^{t-1} f_k(x_i) + f_t(x_i)
$$

ここで、$t$ ステップのときの目的関数は以下のように書ける：

$$
\begin{eqnarray}
\text{obj}^{(t)} &=&  \sum_{i=1}^n l(y_i, \hat{y}_i^{(t)}) + \sum_{i=1}^t \Omega(f_i) \\
&=& \sum_{i=1}^n l \left(y_i, \hat{y}_i^{(t-1)} + f_t(x_i) \right) + \sum_{i=1}^t \Omega(f_i) 
\end{eqnarray}
$$

ここで、$y_i$ は訓練データセットからすでに知っている目標変数であり、$\hat{y}_i^{(t-1)}$ は $t-1$ ステップ目までのツリーの線形結合による予測値、$f_t(x_i)$ はこのステップで追加したツリーである。上手いツリーを追加することができれば、目標変数と予測値との誤差 $l$ が小さくなり目的関数の最適化が実現に近づく。

## Taylor expansion

ここで目的関数を、数学的トリックを使ってより解釈しやすいようにする。より正確には、従来使用されている最適化手法を用いるために、目的関数をユークリッド領域の関数に変換する必要があるからである。$f(x)$ の $x_0$ 周りのテイラー展開は：

$$
f(x) = f(x_0) + f^\prime(x_0)(x-x_0)+ \frac{1}{2} f^{\prime\prime}(x_0)(x-x_0)^2 + ...
$$

ここで $x-x_0=\Delta x$ とすれば

$$
f(x_0 + \Delta x) = f(x_0) + f^\prime(x_0)\Delta x + \frac{1}{2} f^{\prime\prime}(x_0) \Delta x^2 + ...
$$

上記の目的関数の第一項をテイラー展開すると（$x_0=\hat{y}_i^{(t-1)}$、$\Delta x = f_t(x_i)$）：

$$
l \left( y_i, \hat{y}_i^{(t-1)} + f_t(x_i) \right) \simeq l(y_i, \hat{y}_i^{(t-1)}) ~+~ \frac{\partial}{\partial \hat{y}_i^{(t-1)}} l(y_i, \hat{y}_i^{(t-1)}) f_t(x_i) ~+~ \frac{1}{2}\frac{\partial^2}{\partial^2 \hat{y}_i^{(t-1)}} l(y_i, \hat{y}_i^{(t-1)}) f_t(x_i)^2 + ...
$$

ゆえに目的関数は、

$$
\text{obj}^{(t)} = \sum_{i=1}^n \left[ l(y_i, \hat{y}_i^{(t-1)}) + g_if_t(x_i) + \frac{1}{2} h_if_t^2(x_i) \right] + \Omega(f_t) + \text{Const.}
$$

ここで、$g_i$ と $h_i$ はそれぞれ、損失関数の１階・２階微分（勾配）である。

$$
\begin{eqnarray}
g_i &=& \partial_{\hat{y}_i^{(t-1)}}~l \left(y_i, \hat{y}_i^{(t-1)} \right) \\
h_i &=& \partial^2_{\hat{y}_i^{(t-1)}}~l \left(y_i, \hat{y}_i^{(t-1)} \right) \\
\end{eqnarray}
$$

ステップ $t$ において、$t-1$ までのステップで計算されているものは定数として扱えるので、定数項を除外すると以下のようになる（正則化項も $t$ だけを取り出していることに留意）：

$$
\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_if_t(x_i) + \frac{1}{2} h_if_t^2(x_i) \right] + \Omega(f_t)
$$

勾配 $g_i,h_i$ の値自体はステップ $t-1$ までで計算されている係数として考えることができるので、この目的関数の値を左右する量は $f_t(x_i)$ = ツリーをどう設計するか、だけである。

## Split finding

ここで、$j$ 番目のリーフに分類されたデータのインデックス集合を $I_j = \{ i|q(x_i)=j \}$ で表す。$q(x)$ はデータ $x$ がどのリーフに分類されたかを返す関数である。これによって、目的関数は、リーフに関するインデックスの和として次のようにまとめることができる：

$$
\begin{eqnarray}
\mathcal{L}^{(t)} &=& \sum_{i=1}^n \left[ q_if_t(x_i) + \frac{1}{2}h_if_t^2 \right] + \gamma T + \frac{1}{2} \sum_{j=1}^T w_j^2 \\
&=& \sum_{j=1}^T \left[ \left(\sum_{i\in I_j}q_i\right)w_j + \frac{1}{2} \left(\sum_{i\in I_j} h_i + \lambda\right)w_j^2\right] + \gamma T
\end{eqnarray}
$$

固定された構造 $q(x)$ に関して、$j$ 番目のリーフの重みの最適解は

$$
w_j^* = \frac{\sum_{i\in I_j}q_i}{\sum_{i\in I_j} h_i + \lambda}
$$

また最適な目的関数は、

$$
\mathcal{L}^{(t)} (q) = -\frac{1}{2} \sum_{j=1}^T \frac{(\sum_{i\in I_j}q_i)^2}{\sum_{i\in I_j} h_i + \lambda} + \gamma T
$$

と表される。整理すると

- $g_i, h_i$ は定数係数であり、$t-1$ ステップまでの試行で決定されているものである
- $\lambda$ も定数係数であり、一種のhyperparameter である
- そのため、最適解 $w_j^*$ は構造 $q$ の関数である


しかし、ここで問題なのが、全ての木構造 $q$ に関して走査することが通常は不可能であるということである。その代わり、「greedy algorithm（貪欲法）」を使って、single leaf （これはroot nodeとして解釈できる）から初めて枝を追加していく方法が取られる。一度スプリットすると、$I_L$ と $I_R$（全てのサンプルが左か右のノードに分かれる）という状態ができ、$I = I_L \cup I_R$ であるとする。これを用いることで、もともとの損失関数の値からどう変化した（loss reduction） が計算することができる：

$$
\mathcal{L}_{\text{split}} = \frac{1}{2} \left[ 
\frac{(\sum_{i\in I_L}q_i)^2}{\sum_{i\in I_L} h_i + \lambda}
+\frac{(\sum_{i\in I_R}q_i)^2}{\sum_{i\in I_LR} h_i + \lambda}
- \frac{(\sum_{i\in I}q_i)^2}{\sum_{i\in I} h_i + \lambda}
\right]
- \gamma
$$